# III. Linear codes for Inner Product Masking

### $n=3$ shares, $\ell=4$ bits

Parameters:

- $Z=(X + L_2Y_2 + L_3Y_3, Y_2, Y_3)=X\mathbf{G} + Y\mathbf{H}$ where $X, Y=(Y_2, Y_3)$ and $Z$ are the sensitive variable, a mask and the protected variable, respectively. $\mathbf{G} = [1, 0, 0]$ and $\mathbf{H} = [[L_2, 1, 0], [L_3, 0, 1]]$ are two generator matrices of codes $\mathcal{C}$ and $\mathcal{D}$, resp.
- $L_2, L_3\in \mathbb{F}_{2^\ell}\backslash\{0\}$, thus there are 15*15=225 linear codes for IPM
- Each element over $\mathbb{F}_{2^\ell}$ can be denoted as $\alpha^i$ where $i\in\{0, 1, \ldots, 14\}$
- $\mathbf{H}^\perp = [1, L_2, L_3]$ is the generator matirx of dual code $\mathcal{D}^\perp$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd # Pandas for tables
from IPython.display import Latex
from IPython.display import HTML

In [2]:
def read_f(file_name):
    """Reading weight enumerators."""
    with open(file_name, 'r') as fp:
        wd = fp.read().split("]\n")[:-1] # "\n"
        wd = np.array([list(map(int, re.findall(r"\d+", elem))) for elem in wd])
        
    return wd

## 1. Loading all weight enumerators

In [3]:
wd = read_f("weight_distrib_n3k4.txt") # Weight distribution

# print(wd.shape) # 226 entries: 225 for IPM codes and one for BKLC codes

### 1.1 Generating values

In [4]:
alpha_all = np.array(['$\\alpha^{%d}$'%i for i in np.arange(15)])
d_all = np.zeros(len(wd))
B_all = np.zeros(len(wd))
L_2 = np.zeros(len(wd), dtype=int)
L_3 = np.zeros(len(wd), dtype=int)
for i in range(len(wd)):
    d_all[i] = wd[i][2]
    B_all[i] = wd[i][3]
    L_2[i] = int(i/15)
    L_3[i] = int(i%15)

### 1.2 Defining styles of dataframe

See more setting of dataframe from https://mode.com/example-gallery/python_dataframe_styling/

In [5]:
# Set properties for th, td and caption elements in dataframe
th_props = [('font-size', '14px'), ('text-align', 'left'), ('font-weight', 'bold'), ('background-color', '#E0E0E0')]
td_props = [('font-size', '13px'), ('text-align', 'left'), ('min-width', '80px')]
cp_props = [('font-size', '16px'), ('text-align', 'center')]
# Set table styles
styles = [dict(selector="th", props=th_props), dict(selector="td", props=td_props), dict(selector="caption", props=cp_props)]
cm_1 = sns.light_palette("red", as_cmap=True)
cm_2 = sns.light_palette("purple", as_cmap=True, reverse=True)

In [6]:
df = pd.DataFrame({'$L_2$': alpha_all[L_2[:-1]], '$L_3$': alpha_all[L_3[:-1]], '$d_{\mathcal{D}}^\perp$': d_all[:-1], 
                   '$B_{d_{\mathcal{D}}^\perp}$': B_all[:-1], 'Weight Enumerators': wd[:-1]})

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 800)
(df.style
    .background_gradient(cmap=cm_1, subset=['$d_{\mathcal{D}}^\perp$','$B_{d_{\mathcal{D}}^\perp}$' ])
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. I All linear codes for IPM with $n=3$ shares over $\mathbb{F}_{2^4}$.')
    .set_table_styles(styles))


,$L_2$,$L_3$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
0,$\alpha^{0}$,$\alpha^{0}$,3,4,"[0, 1, 3, 4, 6, 6, 9, 4, 12, 1]"
1,$\alpha^{0}$,$\alpha^{1}$,3,3,"[0, 1, 3, 3, 4, 1, 5, 1, 6, 3, 7, 2, 8, 2, 9, 1, 10, 1, 11, 1]"
2,$\alpha^{0}$,$\alpha^{2}$,3,2,"[0, 1, 3, 2, 4, 2, 5, 1, 6, 2, 7, 4, 8, 1, 9, 1, 10, 2]"
3,$\alpha^{0}$,$\alpha^{3}$,3,1,"[0, 1, 3, 1, 4, 3, 5, 1, 6, 2, 7, 3, 8, 2, 9, 3]"
4,$\alpha^{0}$,$\alpha^{4}$,4,3,"[0, 1, 4, 3, 5, 2, 6, 2, 7, 4, 8, 2, 9, 2]"
5,$\alpha^{0}$,$\alpha^{5}$,4,3,"[0, 1, 4, 3, 5, 1, 6, 3, 7, 6, 9, 1, 10, 1]"
6,$\alpha^{0}$,$\alpha^{6}$,4,3,"[0, 1, 4, 3, 5, 2, 6, 2, 7, 4, 8, 2, 9, 2]"
7,$\alpha^{0}$,$\alpha^{7}$,4,2,"[0, 1, 4, 2, 5, 4, 6, 1, 7, 4, 8, 3, 10, 1]"
8,$\alpha^{0}$,$\alpha^{8}$,4,2,"[0, 1, 4, 2, 5, 4, 6, 1, 7, 4, 8, 3, 10, 1]"
9,$\alpha^{0}$,$\alpha^{9}$,4,1,"[0, 1, 4, 1, 5, 5, 6, 3, 7, 2, 8, 2, 9, 1, 10, 1]"


## 2. Optimal linear codes for IPM

### 2.1 Linear codes with $d_{\mathcal{D}}^\perp=6$

We focus on the the linear codes with greater $d_{\mathcal{D}}^\perp$, which are better in the sense of side-channel resistance (from our paper).

In [7]:
# Finding the indices of d_C=6
d_index = []
d_index_L2 = []
d_index_L3 = []
d_C = 6
for i in range(len(wd)-1):
    if wd[i][2] == d_C:
        d_index.append(i)
        d_index_L2.append(int(i/15))
        d_index_L3.append(int(i%15))

#d_index = np.array(d_index)

In [8]:
print(d_index)

[85, 155]


In [9]:
def highlight(s, threshold, column):
    is_min = pd.Series(data=False, index=s.index)
    is_min[column] = (s.loc[column] <= threshold)
    return ['background-color: gold' if is_min.any() else '' for v in is_min]

In [12]:
df_4 = pd.DataFrame({'$L_2$': np.array(alpha_all)[d_index_L2], '$L_3$': np.array(alpha_all)[d_index_L3], '$d_{\mathcal{D}}^\perp$': 
                     d_all[d_index], '$B_{d_{\mathcal{D}}^\perp}$': B_all[d_index], 'Weight Enumerators': wd[d_index]})
df_4 = df_4.sort_values(by=['$B_{d_{\mathcal{D}}^\perp}$'], ascending=True)

(df_4.style
    .apply(highlight, threshold=12, column=['$B_{d_{\mathcal{D}}^\perp}$'], axis=1)
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. II Linear codes for IPM with $d_{\mathcal{D}}^\perp=6$.')
    .set_table_styles(styles))

,$L_2$,$L_3$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
0,$\alpha^{5}$,$\alpha^{10}$,6,12,"[0, 1, 6, 12, 8, 3]"
1,$\alpha^{10}$,$\alpha^{5}$,6,12,"[0, 1, 6, 12, 8, 3]"


### 2.2 Optimal codes for IPM

As shown in our paper, the codes satifying two conditions are optimal:

- Maximizing $d_{\mathcal{D}}^\perp$, here $\max\{d_{\mathcal{D}}^\perp\} = 6$
- Minimizing $B_{d_{\mathcal{D}}^\perp}$, here $\min\{B_{d_{\mathcal{D}}^\perp}\} = 12$

Note that we use two leakage detection metrics **SNR** (signal-to-noise ratio) and **MI** (mutual information), and one leakage exploitation metric **SR** (success rate) to assess the side-channel resistance of IPM with different codes.

As a result of Tab. II, we conclude that the optimal codes for IPM are genetated by $\mathbf{H}=[[L_2, 1, 0], [L_3, 0, 1]]$ where $(L_2, L_3)\in\{(\alpha^5, \alpha^{10}), (\alpha^{10}, \alpha^{5})\}$. Note that these two codes are equivalent.

### 2.3 BKLC code with parameter $[12, 4, 6]$

BKLC is the short of Best Known Linear Code. Note that the code $[12, 4, 6]$ is unique.

In [11]:
bklc_index = [-1]
cm_3 = sns.light_palette("red", as_cmap=True, reverse=True)
df_bklc = pd.DataFrame({'$L_2$': ['  --'], '$L_3$': ['  --'], '$d_{\mathcal{D}}^\perp$': d_all[-1], '$B_{d_{\mathcal{D}}^\perp}$': 
                        B_all[-1], 'Weight Enumerators': wd[bklc_index]})

(df_bklc.style
    .background_gradient(cmap=cm_3, subset=['$d_{\mathcal{D}}^\perp$', '$B_{d_{\mathcal{D}}^\perp}$'])
    .set_caption('Tab. III A BKLC code for IPM with $d_{\mathcal{D}}^\perp=6$.')
    .set_table_styles(styles))

,$L_2$,$L_3$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
0,--,--,6,12,"[0, 1, 6, 12, 8, 3]"


We can see that this BKLC code is equivalent to the best linear codes in IPM. It is interesting to notice that the BKLC code $[12, 4, 6]$ can be generated by $\mathbf{H}^\perp=[1, L_2=\alpha^5, L_3=\alpha^{10}]$.